In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.autonotebook import tqdm

In [ ]:
df=pd.read_csv('/content/drive/My Drive/Colab Notebooks/project Question Pair Similarity/quora_train.csv');


In [ ]:
df.head()

In [ ]:
df.info()


In [ ]:
si=len(df)

In [ ]:
si

In [ ]:
ans=df.groupby('is_duplicate')['id'].count()

In [ ]:
print("percentage of question pairs that are similar",(round(ans[0]/si,2))*100,"%")

In [ ]:
print("percentage of question pairs that are similar",(round(ans[1]/si,2))*100,"%")

In [ ]:
bo=df[df.duplicated()]

In [ ]:
len(bo)

<p> There are no duplicates </p> 

In [ ]:
qid=pd.Series(df['qid1'].tolist()+df['qid2'].tolist())


In [ ]:
print("Total no of unique questions:",len(np.unique(qid)))

In [ ]:
uni=qid.value_counts()
print(uni)

In [ ]:
len(uni)

In [ ]:
plt.figure(figsize=(17, 8))
plt.hist(uni,bins=160)
plt.yscale('log', nonposy='clip')
plt.plot()

Checking for the null values


In [ ]:
row=df[df.isnull().any(1)]   

In [ ]:
row

In [ ]:
df = df.fillna('')
ro=df[df.isnull().any(1)]

In [ ]:
ro

<h1> <b>Adding some Basic Features</b> </h1>

- **frequency of questino 1 occurs**

In [ ]:
df['freq_qid1'] = df.groupby('qid1')['qid1'].transform('count') 

- **frequency of question 2 occurs**

In [ ]:
df['freq_qid2']=df.groupby('qid2')['qid2'].transform('count')

- **length of question 1**

In [ ]:
df['q1_len']=df['question1'].transform(func =lambda s: len(s) )

- **length of question 2**

In [ ]:
df['q2_len']=df['question2'].transform(func=lambda s: len(s))

- **no of words in question 1**

In [ ]:
df['q1_wordscount']=df['question1'].transform(func=lambda s:len(s.split(" ")))

- **no of words in question 2**


In [ ]:
df['q2_wordscount']=df['question2'].transform(func=lambda s:len(s.split(" ")))

- **no of common words between queston 1 and question 2**

In [ ]:
def word_common(row):
  q1=row['question1'].lower()
  q2=row['question2'].lower()
  q1=set(q1.split(" "))
  q2=set(q2.split(" "))
  return len(q1&q2)
df['common_words_count']=df.apply(word_common,axis=1)


- **total words count**

In [ ]:
def cou(row):
  return len(set(row['question1'].split()))+len(set(row['question2'].split()))
df['tot_words']=df.apply(cou,axis=1)

- **words ratio =(common_words_count/tot_words)**

In [ ]:
def rati(row):
  return row['common_words_count']/row['tot_words']
df['words_ratio']=df.apply(rati,axis=1)

- **total frequency of question 1 and question 2 add_freq=freq_qid1+freq_qid2**

In [ ]:
def addfreq(row):
  return (row['freq_qid1']+row['freq_qid2'])
df['add_freq']=df.apply(addfreq,axis=1)


- **subraction of frequency of question 1 and question 2 sub_freq=abs(freq_qid1-freq_qid2)**

In [ ]:
df['sub_freq']=df.apply(lambda row:abs(row['freq_qid1']-row['freq_qid2']),axis=1)


In [ ]:
df

**<h1> Feature Analysis </h1>**

In [ ]:
import seaborn as sns



In [ ]:

def plo(var1,var2):
  plt.figure(figsize=(12,8))
  plt.subplot(1,2,1)
  sns.violinplot(x=var1,y=var2,data=df)
  plt.subplot(1,2,2)
  sns.distplot(df[df[var1]==1][var2],label=1,color='red')
  sns.distplot(df[df[var1]==0][var2],label=0,color='blue')
  plt.legend()
  plt.plot()



In [ ]:
plo('is_duplicate','freq_qid1')

In [ ]:
plo('is_duplicate','freq_qid2')

In [ ]:
plo('is_duplicate','q1_len')

In [ ]:
plo('is_duplicate','q2_len')

In [ ]:
plo('is_duplicate','q1_wordscount')

In [ ]:
plo('is_duplicate','q2_wordscount')

In [ ]:
plo('is_duplicate','common_words_count')

In [ ]:
plo('is_duplicate','tot_words')

In [ ]:
plo('is_duplicate','words_ratio')

In [ ]:
plo('is_duplicate','add_freq')

In [ ]:
plo('is_duplicate','sub_freq')

In [ ]:
df[df.isnull().any(1)]

In [ ]:
df

In [ ]:
df.to_pickle("/content/drive/My Drive/Colab Notebooks/project Question Pair Similarity/basic_features.pkl")

In [ ]:
df = pd.read_pickle("/content/drive/My Drive/Colab Notebooks/project Question Pair Similarity/basic_features.pkl")

**<h1>preprocessing of text data</h1>**

- Stemming words in sentence 
- removing stop words
- Expanding Contractions
- lowering the casese


In [ ]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup
from tqdm.autonotebook import tqdm

porter=PorterStemmer()

In [ ]:
def replac(x):
  x=x.replace("$","dollar")
  x=x.replace("€","euro")
  x=x.replace("₹","rupee")
  x=x.replace("¥","yen")
  x=x.replace("£","pound")
  x=x.replace("%","percent")
  x=x.replace("&","and")
  x=x.replace("'ll"," will")
  x=x.replace(","," ")
  x=x.replace("?"," ")
  x=x.replace("i'm","i am")
  x=x.replace("'s"," is")
  x=x.replace("'ve"," have")
  x=x.replace("won't","will not")
  x=x.replace("can't","can not")
  x=x.replace("n't"," not")
  x=x.replace(",000","k")
  x=x.replace("000,000","m")
  x=x.replace("000,000,000","b")
  x=x.replace("."," ")
  return x;

def preprocessing(x):
  x=x.lower()
  x=replac(x)
  x=BeautifulSoup(x)
  x=x.get_text()
  lis=x.split()
  
  st=""
  for word in lis:
    st+=porter.stem(word)
    st+=" "
  x=st;
  return x;

In [ ]:
tqdm.pandas()
df['question1']=df['question1'].progress_apply(preprocessing)
df['question2']=df['question2'].progress_apply(preprocessing)


In [ ]:
df

In [ ]:
df.to_pickle("/content/drive/My Drive/Colab Notebooks/project Question Pair Similarity/preprocessing.pkl")

In [ ]:
df=pd.read_pickle("/content/drive/My Drive/Colab Notebooks/project Question Pair Similarity/preprocessing.pkl")


<h1> Adding NLP and Fuzzy Based Features </h1>


In [ ]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

In [ ]:
stop_words=stopwords.words("english")
print(stop_words)

- cwcmin=common word count/min(no of words in q1,no of words in q2)

In [ ]:
tqdm.pandas()
def cwc_min_cal(row):
  que1=row['question1'].split();
  que2=row['question2'].split();
  que1_words=set(que1).difference(set(stop_words))
  que2_words=set(que2).difference(set(stop_words))
  common_words=que1_words.intersection(que2_words)
  return len(common_words)/(min(len(que1_words),len(que2_words))+0.0001)

df['cwc_min']=df.progress_apply(cwc_min_cal,axis=1)

- cwcmax=common word count/max(no of words in q1,no of words in q2)

In [ ]:
tqdm.pandas()
def cwc_max_cal(row):
  que1=row['question1'].split();
  que2=row['question2'].split();
  que1_words=set(que1).difference(set(stop_words))
  que2_words=set(que2).difference(set(stop_words))
  common_words=que1_words.intersection(que2_words)
  return len(common_words)/(max(len(que1_words),len(que2_words))+0.0001)

df['cwc_max']=df.progress_apply(cwc_max_cal,axis=1)

- cscmin= common stop word count/min(no of stop words in q1,no of stop words in q2)

In [ ]:
tqdm.pandas()
def csc_min_cal(row):
  que1=row['question1'].split();
  que2=row['question2'].split();
  que1_stop_words=set(que1).intersection(set(stop_words))
  que2_stop_words=set(que2).intersection(set(stop_words))
  common_stop_words=que1_stop_words.intersection(que2_stop_words)
  return len(common_stop_words)/(min(len(que1_stop_words),len(que2_stop_words))+0.0001)
df['csc_min']=df.progress_apply(csc_min_cal,axis=1)

- cscmax= common stop word count/max(no of stop words in q1,no of stop words in q2)

In [ ]:
tqdm.pandas()
def csc_max_cal(row):
  que1=row['question1'].split();
  que2=row['question2'].split();
  que1_stop_words=set(que1).intersection(set(stop_words))
  que2_stop_words=set(que2).intersection(set(stop_words))
  common_stop_words=que1_stop_words.intersection(que2_stop_words)
  return len(common_stop_words)/(max(len(que1_stop_words),len(que2_stop_words))+0.0001)
df['csc_max']=df.progress_apply(csc_max_cal,axis=1)

- ctcmin=common token count/min(no of tokens in q1,no of tokens in q2)

In [ ]:
tqdm.pandas()
def ctc_min_cal(row):
  que1=row['question1'].split();
  que2=row['question2'].split();
  common_token=set(que1).intersection(set(que2))
  return len(common_token)/(min(len(que1),len(que2))+0.0001)
df['ctc_min']=df.progress_apply(ctc_min_cal,axis=1)

- ctcmax=common token count/max(no of tokens in q1,no of tokens in q2)

In [ ]:
tqdm.pandas()
def ctc_max_cal(row):
  que1=row['question1'].split();
  que2=row['question2'].split();
  common_token=set(que1).intersection(set(que2))
  return len(common_token)/(max(len(que1),len(que2))+0.0001)
df['ctc_max']=df.progress_apply(ctc_max_cal,axis=1)

- first_word_equal= 1 if first word of both questions are equal else 0

In [ ]:
tqdm.pandas()
def fir_word_eq_cal(row):
  que1=row['question1'].split();
  que2=row['question2'].split();
  if(len(que1)==0 or len(que2)==0):
    return 0.0
  if(que1[0]==que2[0]):
    return 1.0;
  else:
    return 0.0;
df['fir_word_eq']=df.progress_apply(fir_word_eq_cal,axis=1)

- last_word_equal= 1 if last word of both questions are equal else 0

In [ ]:
tqdm.pandas()
def las_word_eq_cal(row):
  que1=row['question1'].split();
  que2=row['question2'].split();
  if(len(que1)==0 or len(que2)==0):
    return 0.0
  if(que1[len(que1)-1]==que2[len(que2)-1]):
    return 1.0;
  else:
    return 0.0;
df['las_word_eq']=df.progress_apply(las_word_eq_cal,axis=1)

- absdiff_tokens=abs(no of tokens in q1- no of tokens in q2)

In [ ]:
tqdm.pandas()
def absdiff_token_cal(row):
  que1=row['question1'].split();
  que2=row['question2'].split();
  return abs(len(que1)-len(que2))
df['absdiff_token']=df.progress_apply(absdiff_token_cal,axis=1)

- mean_token = average (tokens of q1,tokens of q2)

In [ ]:
tqdm.pandas()
def mean_token_cal(row):
  que1=row['question1'].split();
  que2=row['question2'].split();
  return (len(que1)+len(que2))/2

df['mean_token']=df.progress_apply(mean_token_cal,axis=1)

<h3> fuzzy Logic feature </h3>

- fuzz_ratio

In [ ]:
!pip install fuzzywuzzy

In [ ]:
from fuzzywuzzy import fuzz

In [ ]:
tqdm.pandas()
def fuzz_ratio_cal(row):
  return fuzz.ratio(row['question1'],row['question2'])

df['fuzz_ratio']=df.progress_apply(fuzz_ratio_cal,axis=1)

- fuzz_partial_ratio


In [ ]:
tqdm.pandas()
def fuzz_partial_ratio_cal(row):
  return fuzz.partial_ratio(row['question1'],row['question2'])
df['fuzz_partial_ratio']=df.progress_apply(fuzz_partial_ratio_cal,axis=1)


- fuzz_sort_ratio

In [ ]:
tqdm.pandas()
def fuzz_sort_ratio_cal(row):
  return fuzz.token_sort_ratio(row['question1'],row['question2'])
df['fuzz_sort_ratio']=df.progress_apply(fuzz_sort_ratio_cal,axis=1)


- fuzz_set_ratio

In [ ]:
tqdm.pandas()
def fuzz_set_ratio_cal(row):
  return fuzz.token_set_ratio(row['question1'],row['question2'])
df['fuzz_set_ratio']=df.progress_apply(fuzz_set_ratio_cal,axis=1)


- Longest common subsequence ratio



In [ ]:
!pip install distance

In [ ]:
import distance
tqdm.pandas()
def get_longest_substr_ratio(a, b):
    strs = list(distance.lcsubstrings(a, b))
    if len(strs) == 0:
        return 0
    else:
        return len(strs[0]) / (min(len(a), len(b)) + 1)
df["lcs_ratio"]  = df.progress_apply(lambda x: get_longest_substr_ratio(x["question1"], x["question2"]), axis=1)


In [ ]:
df.head(1)

In [ ]:
df.to_pickle("/content/drive/My Drive/Colab Notebooks/project Question Pair Similarity/advanced_features.pkl")

In [ ]:
df=pd.read_pickle("/content/drive/My Drive/Colab Notebooks/project Question Pair Similarity/advanced_features.pkl")

<h1> Fuzzy Feature Analysis </h1>

In [ ]:
plo('is_duplicate','fuzz_ratio')

In [ ]:
plo('is_duplicate','fuzz_partial_ratio')

In [ ]:
plo('is_duplicate','fuzz_sort_ratio')

In [ ]:
plo('is_duplicate','fuzz_set_ratio')

<h1> Pair Plot of Natural language Processing features </h1>


In [ ]:
fea=['ctc_min','ctc_max','cwc_min','cwc_max']
sns.pairplot(df[['ctc_min','ctc_max','cwc_min','cwc_max','is_duplicate']][:],hue='is_duplicate',vars=fea)
plt.show()

<h1> Plotting Word Cloud </h1>

In [ ]:
from wordcloud import WordCloud
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
stop_words=stopwords.words("english")

In [ ]:
df_dup=df[df['is_duplicate']==1]
df_ndup =df[df['is_duplicate'] == 0]


In [ ]:
du = np.dstack([df_dup["question1"], df_dup["question2"]]).flatten()
ndu = np.dstack([df_ndup["question1"], df_ndup["question2"]]).flatten()


In [ ]:
strdu=" ".join(list(du))
strndu=" ".join(list(ndu))


In [ ]:
wc=WordCloud(stopwords=stop_words)
wc.generate(strdu)
plt.imshow(wc, interpolation='bilinear')
plt.axis("off")
plt.show() 

In [ ]:
wc=WordCloud(stopwords=stop_words )
wc.generate(strndu)
plt.imshow(wc, interpolation='bilinear')
plt.axis("off")
plt.show()

<h1> visualization (TSNE) </h1>

In [ ]:
df_sample=df[0:10000]
df_sample.head(1)

In [ ]:
from sklearn.manifold import TSNE
tsne=TSNE(n_components=2,perplexity=30,n_iter=1000)
tsne_df=pd.DataFrame(tsne.fit_transform(df_sample[['cwc_min','cwc_max','csc_min','csc_max','ctc_min','ctc_max','las_word_eq','fir_word_eq','absdiff_token','mean_token','fuzz_ratio','fuzz_partial_ratio','fuzz_sort_ratio','fuzz_set_ratio','lcs_ratio']]),columns=['fea1','fea2'])
tsne_df['is_duplicate']=df_sample['is_duplicate']
tsne_df


In [ ]:
sns.FacetGrid(tsne_df, hue='is_duplicate',height=10).map(plt.scatter,"fea1","fea2").add_legend()
plt.show()

<h1> Tfidf-word vector <h1>

In [ ]:
import spacy

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
ques=list(df['question1'])+list(df['question2'])

In [ ]:
tfidf=TfidfVectorizer()
spar_mat=tfidf.fit_transform(ques)

In [ ]:
idf_val=dict(zip(tfidf.get_feature_names(),tfidf.idf_))

In [ ]:
len(idf_val)

In [ ]:
nlp = spacy.load('en_core_web_sm')
vectors = []
for question in tqdm(list(df['question1'])):
  doc = nlp(question)
  try:
    vec=np.zeros([len(doc[0].vector)])
  except:
    vec=np.zeros(96)
  for word in doc:
    word_vec= word.vector
    try:
      idf =idf_val[str(word)]
    except:
      idf = 0
    vec += (word_vec* idf)
  vectors.append(vec)
df['q1_w2v_idf'] = list(vectors)

In [ ]:
df

In [ ]:
df.to_pickle("/content/drive/My Drive/Colab Notebooks/project Question Pair Similarity/tfidfw2v_q1.pkl")

In [ ]:
df=pd.read_pickle('/content/drive/My Drive/Colab Notebooks/project Question Pair Similarity/tfidfw2v_q1.pkl')

In [ ]:
df

In [ ]:
nlp = spacy.load('en_core_web_sm')
vectors = []
for question in tqdm(list(df['question2'])):
  doc = nlp(question)
  try:
    vec=np.zeros([len(doc[0].vector)])
  except:
    vec=np.zeros(96)
  for word in doc:
    word_vec= word.vector
    try:
      idf =idf_val[str(word)]
    except:
      idf = 0
    vec += (word_vec* idf)
  vectors.append(vec)
df['q2_w2v_idf'] = list(vectors)

In [ ]:
df.to_pickle("/content/drive/My Drive/Colab Notebooks/project Question Pair Similarity/tfidfw2v.pkl")

In [ ]:
df=pd.read_pickle("/content/drive/My Drive/Colab Notebooks/project Question Pair Similarity/tfidfw2v.pkl")

In [ ]:
df

In [ ]:
dfq1=pd.DataFrame((df['q1_w2v_idf'].values).tolist(),index=df.index,columns=["q1_"+str(i) for i in range(len(df['q1_w2v_idf'][0]))])
dfq2=pd.DataFrame((df['q2_w2v_idf'].values).tolist(),index=df.index,columns=["q2_"+str(i) for i in range(len(df['q2_w2v_idf'][0]))])
dfq1['id']=df.id
dfq2['id']=df.id
dfq1q2  = dfq1.merge(dfq2, on='id',how='left')
df=df.merge(dfq1q2,on='id',how='left')
df.drop(['id','q1_w2v_idf','q2_w2v_idf','qid1','qid2','question1','question2'],axis=1,inplace=True)

In [ ]:
df.to_pickle("/content/drive/My Drive/Colab Notebooks/project Question Pair Similarity/final_df.pkl")

In [ ]:
df=pd.read_pickle("/content/drive/My Drive/Colab Notebooks/project Question Pair Similarity/final_df.pkl")

In [ ]:
# df=df[0:400000]

In [ ]:
df.head()

In [ ]:
x_label=df.iloc[:,1:]
y_label=df.iloc[:,0:1]

In [ ]:
y_val=list(map(int,y_label.values))

<h1>Train Test Splitting in stratified Way</h1>

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x_label,y_val,stratify=y_val,test_size=0.3) 

<h1> Training a Random Model

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics.classification import log_loss

In [ ]:
y_predict=np.zeros((len(y_test),2))
for i in range(len(y_test)):
    rand_val= np.random.rand(1,2)
    y_predict[i] = ((rand_val/sum(sum(rand_val)))[0])
#print(Y_predict)
print("Log loss on Test Data using Random Model",log_loss(y_test, y_predict, eps=1e-15))

In [ ]:
y_predict=np.argmax(y_predict,axis=1)
confusion_matrix(y_test,y_predict)

<h1> Logistic Regression </h1>


In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.calibration import CalibratedClassifierCV

In [ ]:
mi=100000
best_alpha=-1;
alpha=[10**i for i in range(-5,2)]
for i in tqdm(alpha):
  model=SGDClassifier(alpha=i,random_state=42 ,loss='log') 
  #model.fit(x_train,y_train)
  sig_model=CalibratedClassifierCV(model,method='sigmoid')
  sig_model.fit(x_train,y_train)
  y_predict=sig_model.predict_proba(x_test)
  logloss=log_loss(y_test,y_predict)
  if(mi>logloss):
    mi=logloss
    best_alpha=i
  print("alpha->",i,"\t\tlog-loss->",logloss)

In [ ]:
best_alpha=0.001

In [ ]:
best_model=SGDClassifier(alpha=best_alpha,random_state=42,loss="log")
best_model=CalibratedClassifierCV(best_model,method="sigmoid")
best_model.fit(x_train,y_train)
y_predict=best_model.predict_proba(x_train)
print("Train Log Loss -->",log_loss(y_train,y_predict))
y_predict=best_model.predict_proba(x_test)
print("Test Log Loss -->",log_loss(y_test,y_predict))
y_predict=np.argmax(y_predict,axis=1)
confusion_matrix(y_test,y_predict)


In [ ]:
to_best_model

<h1> SVM (Support Vector Machine) </h1>

In [ ]:
mi=100000
best_alpha=-1;
alpha=[10**i for i in range(-5,2)]
for i in tqdm(alpha):
  model=SGDClassifier(alpha=i,random_state=42,loss="hinge")
  #model.fit(x_train,y_train)
  sig_model=CalibratedClassifierCV(model,method='sigmoid')
  sig_model.fit(x_train,y_train)
  y_predict=sig_model.predict_proba(x_test)
  logloss=log_loss(y_test,y_predict)
  if(mi>logloss):
    mi=logloss
    best_alpha=i
  print("alpha->",i,"\t\tlog-loss->",logloss)

In [ ]:
best_model=SGDClassifier(alpha=best_alpha,random_state=42,loss="hinge")
best_model=CalibratedClassifierCV(best_model,method="sigmoid")
best_model.fit(x_train,y_train)
y_predict=best_model.predict_proba(x_train)
print("Train Log Loss -->",log_loss(y_train,y_predict))
y_predict=best_model.predict_proba(x_test)
print("Test Log Loss -->",log_loss(y_test,y_predict))
y_predict=np.argmax(y_predict,axis=1)
confusion_matrix(y_test,y_predict)


<h1> XGBoost </h1>

In [ ]:
import xgboost as xgb   

In [ ]:
params = {}
params['objective'] = 'binary:logistic'
params['eval_metric'] = 'logloss'
params['eta'] = 0.02
params['max_depth'] = 4
d_train=xgb.DMatrix(x_train,label=y_train)
d_test=xgb.DMatrix(x_test,label=y_test)
watchlist = [(d_train, 'train'), (d_test, 'valid')]
bst = xgb.train(params, d_train, 400, watchlist, early_stopping_rounds=20, verbose_eval=10)
xgdmat = xgb.DMatrix(x_train,y_train)
y_predict = bst.predict(d_test)
print("The test log loss is:",log_loss(y_test, y_predict, eps=1e-15))

In [ ]:
y_predict=np.array(y_predict>0.5,dtype=int)
confusion_matrix(y_test,y_predict)